# Import library:

In [13]:
import numpy as np 
import pandas as pd
import pickle
import itertools



# Uploading the co-occurrence matrix

In [ ]:
all_matrix = np.load("Data/Co_Occurence.npy")
all_matrix

# list of seed gene

In [5]:
GBM = ['EGFR', 'TP53', 'PTEN', 'RB1', 'NF1', 'ATRX', 'PIK3R1', 'BRAF', 'IDH1', 'PIK3CA', 'CDKN2A', 'MDM2', 'BAX', 'ERBB2', 'IDH2', 'PTPN11', 'TERT']
Blood = ['AP3B1', 'FGA', 'PROS1', 'ITGA2B', 'F13A1', 'F10', 'WAS', 'GGCX', 'F5', 'ITGB3', 'F7', 'F9', 'JAK2', 'LMAN1', 'BLOC1S3', 'CFH', 'MCFD2', 'DTNBP1', 'F8', 'BLOC1S6', 'GP9', 'GP1BB', 'FGB', 'PROC', 'F2', 'ADAMTS13', 'CD36', 'F13B', 'VWF', 'CFD', 'THPO', 'HPS6', 'HPS5', 'SERPINC1', 'FGG', 'F11', 'F12']
peroxisomal = ['ABCD1', 'PEX5', 'PEX1', 'MVK', 'PEX19', 'CAT', 'PEX13', 'PEX26', 'PEX2', 'PEX7', 'GNPAT', 'PEX12', 'PHYH', 'AGPS', 'PEX6', 'PEX14', 'ACOX1', 'PEX10', 'PEX3', 'PEX16']
cardiomo = ['TTN', 'TNNT2', 'PLN', 'PRKAG2', 'MYH7', 'TPM1', 'JPH2', 'MYO6', 'ACTC1', 'MYL3', 'TNNI3', 'VCL', 'TNNC1', 'CAV3', 'MYBPC3', 'NEXN', 'TCAP', 'MYOZ2', 'MYL2', 'CSRP3']
lysosmal = ['AGA', 'MAN2B1', 'APOE', 'PHKB', 'GYS1', 'GBA', 'GLB1', 'GM2A', 'PHKG2', 'CTSK', 'GNE', 'GUSB', 'SMPD1', 'GAA', 'CTSA', 'PHKA2', 'MCOLN1', 'HEXA', 'PSAP', 'HEXB', 'ARSA', 'GNPTG', 'PHKA1', 'PGM1', 'NPC2', 'ASAH1', 'NEU1', 'IDUA']
sarcoma = ['NCOA1', 'COL1A1', 'CHEK2', 'EXT1', 'ETV1', 'FOXO1', 'TFE3', 'JAZF1', 'ERG', 'EWSR1', 'WT1', 'SS18', 'FLI1', 'PAX7', 'SUZ12', 'DDIT3', 'PDGFB', 'FUS', 'SSX2', 'NR4A3', 'PAX3', 'RB1']


# Uploading index_to_vocab file for matrix

In [6]:
df_temp = pd.read_csv("Data/index_to_vocab.csv")

vocab =list(df_temp['vocab'])
index_vocab = list(df_temp['index'])

index_to_vocab = dict(zip(index_vocab,vocab))
vocab_to_index = dict(zip(vocab,index_vocab))

# Uploading frequency of any gene in all sentences

In [8]:
a_file = open("Data/frequency_ofgene.pkl", "rb")
frequency_of_gene = pickle.load(a_file)
a_file. close()

# Disease module identification function 

In [9]:
def DMI(Seed_genes,CO_matrix,Gene_freq,output_size):
    size_of_seed_genes = len(Seed_genes)
    sorted_C_prob_seed_gene_holder = list()
    matrix_index_of_seed_gene = [vocab_to_index[i] for i in Seed_genes]
    for seed_index in range(0,size_of_seed_genes):
        Seed_gene_vector = CO_matrix[vocab_to_index[Seed_genes[seed_index]]]
        C_prob_seed_gene = {index_to_vocab[i]:(Seed_gene_vector[i]/(Gene_freq[Seed_genes[seed_index]])) for i in range(0,len(Seed_gene_vector)) if i not in matrix_index_of_seed_gene}
        temp = dict(sorted(C_prob_seed_gene.items(), key=lambda item: item[1],reverse=True))
        sorted_C_prob_seed_gene_holder.append(temp)
        
    
    ext_gene_name = []
    for i in range(0,size_of_seed_genes):
        ext_gene_name +=list(sorted_C_prob_seed_gene_holder[i].keys())[:output_size]
    final_list = {i:0 for i in set(ext_gene_name)}    
  
    rank_of_any_gene = {i:[] for i in set(ext_gene_name)}
    
    for j in range(0,size_of_seed_genes):
        for i in range(0,output_size):
            final_list[list(sorted_C_prob_seed_gene_holder[j].keys())[i]]+=list(sorted_C_prob_seed_gene_holder[j].values())[i]
            rank_of_any_gene[list(sorted_C_prob_seed_gene_holder[j].keys())[i]].append(i)
            
    gene_score_variance = {(gene,final_list[gene],Variance_function(rank_of_any_gene[gene])) for gene in list(final_list.keys())}        
    sorted_final =  sorted(gene_score_variance, key = lambda x: (-x[1],-x[2]))
    top_gene = [i[0] for i in sorted_final][:output_size]
    return top_gene
        

In [10]:

def Variance_function(input_list):
    mean = sum(input_list) / len(input_list)
    res = sum((i - mean) ** 2 for i in input_list) / len(input_list)
    return res



In [11]:

def recall_calculation(Seed_gene,Disease_Gene,co_matrix,freq,output):
    recall = 0 
    disease_module = DMI(Seed_gene,co_matrix,freq,output)
    for j in Disease_Gene:
        if j in disease_module:
            recall +=1
    print("Recall of Seed Genes", Seed_gene ,recall/(len(Disease_Gene)-len(Seed_gene)))
    return recall/(len(Disease_Gene)-len(Seed_gene))


data = itertools.combinations(GBM, 3)
my_Set = list(data)

average_racall = []
for i in my_Set:
    average_racall.append(recall_calculation(i,GBM,all_matrix,frequency_of_gene,200))         
print("aveage recall",np.mean(average_racall))

        
    

Recall of Seed Gene ('EGFR', 'TP53', 'PTEN') 0.5
Recall of Seed Gene ('EGFR', 'TP53', 'RB1') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'TP53', 'NF1') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'TP53', 'ATRX') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'TP53', 'PIK3R1') 0.6428571428571429
Recall of Seed Gene ('EGFR', 'TP53', 'BRAF') 0.5
Recall of Seed Gene ('EGFR', 'TP53', 'IDH1') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'TP53', 'PIK3CA') 0.6428571428571429
Recall of Seed Gene ('EGFR', 'TP53', 'CDKN2A') 0.5
Recall of Seed Gene ('EGFR', 'TP53', 'MDM2') 0.5
Recall of Seed Gene ('EGFR', 'TP53', 'BAX') 0.5
Recall of Seed Gene ('EGFR', 'TP53', 'ERBB2') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'TP53', 'IDH2') 0.5
Recall of Seed Gene ('EGFR', 'TP53', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'TP53', 'TERT') 0.5714285714285714
Recall of Seed Gene ('EGFR', 'PTEN', 'RB1') 0.5
Recall of Seed Gene ('EGFR', 'PTEN', 'NF1') 0.5714285714285714
Recall of See

Recall of Seed Gene ('TP53', 'RB1', 'NF1') 0.5714285714285714
Recall of Seed Gene ('TP53', 'RB1', 'ATRX') 0.5
Recall of Seed Gene ('TP53', 'RB1', 'PIK3R1') 0.6428571428571429
Recall of Seed Gene ('TP53', 'RB1', 'BRAF') 0.5
Recall of Seed Gene ('TP53', 'RB1', 'IDH1') 0.5
Recall of Seed Gene ('TP53', 'RB1', 'PIK3CA') 0.6428571428571429
Recall of Seed Gene ('TP53', 'RB1', 'CDKN2A') 0.5
Recall of Seed Gene ('TP53', 'RB1', 'MDM2') 0.5
Recall of Seed Gene ('TP53', 'RB1', 'BAX') 0.42857142857142855
Recall of Seed Gene ('TP53', 'RB1', 'ERBB2') 0.5714285714285714
Recall of Seed Gene ('TP53', 'RB1', 'IDH2') 0.5
Recall of Seed Gene ('TP53', 'RB1', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('TP53', 'RB1', 'TERT') 0.5
Recall of Seed Gene ('TP53', 'NF1', 'ATRX') 0.6428571428571429
Recall of Seed Gene ('TP53', 'NF1', 'PIK3R1') 0.7142857142857143
Recall of Seed Gene ('TP53', 'NF1', 'BRAF') 0.5714285714285714
Recall of Seed Gene ('TP53', 'NF1', 'IDH1') 0.6428571428571429
Recall of Seed Gene ('TP

Recall of Seed Gene ('PTEN', 'PIK3R1', 'PTPN11') 0.6428571428571429
Recall of Seed Gene ('PTEN', 'PIK3R1', 'TERT') 0.7142857142857143
Recall of Seed Gene ('PTEN', 'BRAF', 'IDH1') 0.5714285714285714
Recall of Seed Gene ('PTEN', 'BRAF', 'PIK3CA') 0.6428571428571429
Recall of Seed Gene ('PTEN', 'BRAF', 'CDKN2A') 0.5
Recall of Seed Gene ('PTEN', 'BRAF', 'MDM2') 0.5
Recall of Seed Gene ('PTEN', 'BRAF', 'BAX') 0.5
Recall of Seed Gene ('PTEN', 'BRAF', 'ERBB2') 0.5
Recall of Seed Gene ('PTEN', 'BRAF', 'IDH2') 0.5
Recall of Seed Gene ('PTEN', 'BRAF', 'PTPN11') 0.5
Recall of Seed Gene ('PTEN', 'BRAF', 'TERT') 0.5714285714285714
Recall of Seed Gene ('PTEN', 'IDH1', 'PIK3CA') 0.6428571428571429
Recall of Seed Gene ('PTEN', 'IDH1', 'CDKN2A') 0.5
Recall of Seed Gene ('PTEN', 'IDH1', 'MDM2') 0.5
Recall of Seed Gene ('PTEN', 'IDH1', 'BAX') 0.5
Recall of Seed Gene ('PTEN', 'IDH1', 'ERBB2') 0.5714285714285714
Recall of Seed Gene ('PTEN', 'IDH1', 'IDH2') 0.5714285714285714
Recall of Seed Gene ('PTEN', 'I

Recall of Seed Gene ('NF1', 'PIK3R1', 'IDH1') 0.6428571428571429
Recall of Seed Gene ('NF1', 'PIK3R1', 'PIK3CA') 0.5714285714285714
Recall of Seed Gene ('NF1', 'PIK3R1', 'CDKN2A') 0.7142857142857143
Recall of Seed Gene ('NF1', 'PIK3R1', 'MDM2') 0.7142857142857143
Recall of Seed Gene ('NF1', 'PIK3R1', 'BAX') 0.7142857142857143
Recall of Seed Gene ('NF1', 'PIK3R1', 'ERBB2') 0.7142857142857143
Recall of Seed Gene ('NF1', 'PIK3R1', 'IDH2') 0.5
Recall of Seed Gene ('NF1', 'PIK3R1', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('NF1', 'PIK3R1', 'TERT') 0.6428571428571429
Recall of Seed Gene ('NF1', 'BRAF', 'IDH1') 0.6428571428571429
Recall of Seed Gene ('NF1', 'BRAF', 'PIK3CA') 0.7142857142857143
Recall of Seed Gene ('NF1', 'BRAF', 'CDKN2A') 0.5714285714285714
Recall of Seed Gene ('NF1', 'BRAF', 'MDM2') 0.5714285714285714
Recall of Seed Gene ('NF1', 'BRAF', 'BAX') 0.5714285714285714
Recall of Seed Gene ('NF1', 'BRAF', 'ERBB2') 0.5714285714285714
Recall of Seed Gene ('NF1', 'BRAF', 'IDH2'

Recall of Seed Gene ('PIK3R1', 'PIK3CA', 'MDM2') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'PIK3CA', 'BAX') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'PIK3CA', 'ERBB2') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'PIK3CA', 'IDH2') 0.35714285714285715
Recall of Seed Gene ('PIK3R1', 'PIK3CA', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('PIK3R1', 'PIK3CA', 'TERT') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'CDKN2A', 'MDM2') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'CDKN2A', 'BAX') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'CDKN2A', 'ERBB2') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'CDKN2A', 'IDH2') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'CDKN2A', 'PTPN11') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'CDKN2A', 'TERT') 0.7142857142857143
Recall of Seed Gene ('PIK3R1', 'MDM2', 'BAX') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'MDM2', 'ERBB2') 0.6428571428571429
Recall of Seed Gene ('PIK3R1', 'MDM2', 'IDH2') 

Recall of Seed Gene ('MDM2', 'BAX', 'IDH2') 0.5
Recall of Seed Gene ('MDM2', 'BAX', 'PTPN11') 0.5
Recall of Seed Gene ('MDM2', 'BAX', 'TERT') 0.5
Recall of Seed Gene ('MDM2', 'ERBB2', 'IDH2') 0.5714285714285714
Recall of Seed Gene ('MDM2', 'ERBB2', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('MDM2', 'ERBB2', 'TERT') 0.5714285714285714
Recall of Seed Gene ('MDM2', 'IDH2', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('MDM2', 'IDH2', 'TERT') 0.5714285714285714
Recall of Seed Gene ('MDM2', 'PTPN11', 'TERT') 0.5714285714285714
Recall of Seed Gene ('BAX', 'ERBB2', 'IDH2') 0.5
Recall of Seed Gene ('BAX', 'ERBB2', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('BAX', 'ERBB2', 'TERT') 0.6428571428571429
Recall of Seed Gene ('BAX', 'IDH2', 'PTPN11') 0.5714285714285714
Recall of Seed Gene ('BAX', 'IDH2', 'TERT') 0.5714285714285714
Recall of Seed Gene ('BAX', 'PTPN11', 'TERT') 0.5714285714285714
Recall of Seed Gene ('ERBB2', 'IDH2', 'PTPN11') 0.6428571428571429
Recall of Seed Gene ('